In [ ]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.diagnostic import acorr_ljungbox
import warnings
warnings.filterwarnings('ignore')

def создать_модель_arima_и_вывести_прогноз(путь_к_файлу, имя_столбца_цены):
    """
    Функция прогнозирования с ручным подбором параметров ARIMA,
    выводом исторического прогноза и проверкой остатков.
    Без метрики R², как было запрошено.
    """
    try:
        # 1. Загрузка и подготовка данных
        df = pd.read_csv(путь_к_файлу, parse_dates=['Date'])
        df = df.sort_values(by='Date').set_index('Date')
        
        if имя_столбца_цены not in df.columns:
            raise KeyError(f"Столбец '{имя_столбца_цены}' не найден.")
            
        series = df[имя_столбца_цены].dropna()
        
        # 2. Проверка на стационарность и определение порядка дифференцирования
        print("Проверка стационарности данных...")
        adf_result = adfuller(series)
        print(f'ADF Statistic: {adf_result[0]:.4f}')
        print(f'p-value: {adf_result[1]:.4f}')
        
        # Если ряд не стационарный, применяем дифференцирование
        d = 0
        if adf_result[1] > 0.05:
            d = 1
            series_diff = series.diff().dropna()
            print("Ряд не стационарный, применяем дифференцирование (d=1)")
        else:
            print("Ряд стационарный, дифференцирование не требуется (d=0)")
        
        # 3. Подбор параметров ARIMA (p,d,q)
        p = 2  # Авторегрессионная часть
        q = 1  # Скользящее среднее
        print(f"\nИспользуемые параметры ARIMA: p={p}, d={d}, q={q}")
        
        # 4. Обучение модели
        arima_model = ARIMA(series, order=(p, d, q))
        model_fit = arima_model.fit()
        
        # 5. Прогнозирование для исторического периода
        historical_forecast = model_fit.predict(start=series.index[0], end=series.index[-1])
        
        # 6. Визуализация
        plt.figure(figsize=(14, 7))
        
        # Фактические данные
        plt.plot(series.index, series, label='Фактические данные', 
                color='#1f77b4', linewidth=2)
        
        # Прогноз для исторического периода
        plt.plot(historical_forecast.index, historical_forecast, 
                label='Прогноз модели', color='#2ca02c', 
                linestyle='--', linewidth=2)
        
        # Настройки графика
        plt.title(f'Исторический прогноз цены арматуры (ARIMA({p},{d},{q}))', fontsize=16)
        plt.xlabel('Дата', fontsize=12)
        plt.ylabel('Цена', fontsize=12)
        plt.legend(fontsize=10)
        plt.grid(True, linestyle='--', alpha=0.5)
        plt.tight_layout()
        plt.show()
        
        # 7. Расчет и вывод метрик (без R²)
        mse = mean_squared_error(series, historical_forecast)
        mae = mean_absolute_error(series, historical_forecast)
        rmse = np.sqrt(mse)
        
        print("\nМетрики качества модели:")
        print(f"Среднеквадратичная ошибка (RMSE): {rmse:.4f}")
        print(f"Средняя абсолютная ошибка (MAE): {mae:.4f}")
        
        # Расчет MAPE (с обработкой деления на ноль)
        with np.errstate(divide='ignore', invalid='ignore'):
            mape = np.mean(np.abs((series - historical_forecast) / series)) * 100
            mape = np.nan_to_num(mape, nan=0.0, posinf=0.0, neginf=0.0)
        print(f"Средняя абсолютная процентная ошибка (MAPE): {mape:.2f}%")
        
        # 8. Тест Льюнга-Бокса на автокорреляцию остатков
        residuals = series - historical_forecast
        lb_test = acorr_ljungbox(residuals.dropna(), lags=[10], return_df=True)
        lb_pvalue = lb_test['lb_pvalue'].values[0]
        print(f"\nТест Льюнга-Бокса (p-value): {lb_pvalue:.4f}")
        
        if lb_pvalue > 0.05:
            print("Остатки не имеют значимой автокорреляции (хорошо)")
        else:
            print("Остатки имеют автокорреляцию (плохо)")
    
    except Exception as e:
        print(f"Произошла ошибка: {str(e)}")

# Пример использования
путь_к_файлу = 'cleaned_stationary_data.csv'
имя_столбца_цены = 'Цена на арматуру_x_stationary'
создать_модель_arima_и_вывести_прогноз(путь_к_файлу, имя_столбца_цены)